### Import

In [1]:
import sys

sys.path.append('../')
sys.path.append('../ride/')

from ride_pfa import graph_osm_loader
import ride_pfa.centroid_graph.centroids_graph_builder as cgb
import ride_pfa.path_finding as pfa 
import ride_pfa.clustering as cls
import ride_pfa.utils as utils
import ride_pfa.benchmark as bench
from ride_pfa.centroid_graph import CentroidGraph

### Graph download

In [2]:
id = graph_osm_loader.osm_cities_example['Paris']
g = graph_osm_loader.get_graph(id)

### Extraction Path Finding

In [3]:
# To decrease error in pathfinding better to use inverse weight in Louvain 
utils.add_inverse_edges_weight(g, weight='length', inverse_weight='inverse')

'inverse'

In [4]:
cms_resolver = cls.LouvainKMeansCommunityResolver(resolution=1, weight='inverse', k_means_weight='length')
cg_builder = cgb.CentroidGraphBuilder()

def build_and_get_time() -> tuple[float, CentroidGraph]:
    return utils.get_execution_time(lambda: cg_builder.build(g, cms_resolver), iterations=1)

t, cg = build_and_get_time()
t, len(cg.cms)

find centroids:   0%|          | 0/105 [00:00<?, ?it/s]

find edges:   0%|          | 0/105 [00:00<?, ?it/s]

(57.68914866447449, 105)

In [5]:
baseline = pfa.Dijkstra(g)

In [6]:
test_alg = pfa.ExtractionPfa(
    g = g,
    upper=pfa.Dijkstra(cg.g),
    down=pfa.Dijkstra(g),
    cluster='cluster'
)

In [7]:
points = utils.generate_points(g, num=100)

In [8]:
stat = bench.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=4
).compare()

start 0


find paths:   0%|          | 0/25 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/25 [00:00<?, ?it/s]

start 2


find paths:   0%|          | 0/25 [00:00<?, ?it/s]

start 3


find paths:   0%|          | 0/25 [00:00<?, ?it/s]

In [9]:
stat


        err_max:    51.9194
        err_min:    0.0000
        err_median: 4.7511
        err_mean:   6.1828

        acceleration_max:    16.9445
        acceleration_min:    0.7046
        acceleration_median: 6.3546
        acceleration_mean:   6.8154
        